In [8]:
# Import Dependencies
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import csv
pd.set_option('display.max_rows',50)

In [9]:
# Create a path to the csv and read it into a Pandas DataFrame
#csv_path = "Assessor_Historical_Secured_Property_Tax_Rolls.csv"
csv_path = "combine_updated.csv"
rent_path = "../Rent Data/rent.csv"
home_df = pd.read_csv(csv_path, encoding='utf8', engine='python')
rent_df = pd.read_csv(rent_path, encoding='utf8', engine='python')

In [10]:
#Changing from columns to rows
my_col_names = list(rent_df.columns.values)[4:11]
melted_df = pd.melt(rent_df,id_vars=["Region Name","Region Type","Data Type"],
                    value_vars=my_col_names, var_name="Year",value_name="Yearly Rent Price")

In [11]:
melted_df.head()

,Region Name,Region Type,Data Type,Year,Yearly Rent Price
0,Bayview,neighborhood,All Homes,2011,30723
1,Bernal Heights,neighborhood,All Homes,2011,34471
2,Buena Vista,neighborhood,All Homes,2011,42407
3,Corona Heights,neighborhood,All Homes,2011,41051
4,Cow Hollow,neighborhood,All Homes,2011,52856


In [12]:
#changing the type 
melted_df["Year"] = melted_df["Year"].astype("int64")

In [13]:
#merging two data frames
rent_df2 = pd.merge(melted_df,rent_df.loc[:,["Region Name","Region Type",
                                             "Data Type","Avg Rent Rate","City","Lat","Lng"]])

In [14]:
rent_df2.head()

,Region Name,Region Type,Data Type,Year,Yearly Rent Price,Avg Rent Rate,City,Lat,Lng
0,Bayview,neighborhood,All Homes,2011,30723,37087.571429,San Francisco,37.730416,-122.384424
1,Bayview,neighborhood,All Homes,2012,28821,37087.571429,San Francisco,37.730416,-122.384424
2,Bayview,neighborhood,All Homes,2013,30433,37087.571429,San Francisco,37.730416,-122.384424
3,Bayview,neighborhood,All Homes,2014,35338,37087.571429,San Francisco,37.730416,-122.384424
4,Bayview,neighborhood,All Homes,2015,42870,37087.571429,San Francisco,37.730416,-122.384424


In [15]:
#rename columns to merge on same names
rent_df2 = rent_df2.rename(columns={'Region Name': 'Neighborhood'})
home_df = home_df.rename(columns={'Assessor Neighborhood': 'Neighborhood', 'Closed Roll Year': 'Year'})

In [16]:
#merging
merged_csv = pd.merge(home_df,rent_df2, 
                  how='inner',
                  left_on=["Neighborhood","Year"],
                  right_on=["Neighborhood","Year"])

In [21]:
merged_gb= merged_csv.groupby("Neighborhood").mean()

In [22]:
merged_gb

,Unnamed: 0,Year,Assessed Land Value,Pct Change,Yearly Rent Price,Avg Rent Rate,Lat,Lng
Neighborhood,,,,,,,,
Bayview,6.5,2013.5,1.905129e+05,4.170000,35644.333333,37087.571429,37.730416,-122.384424
Bernal Heights,16.5,2013.5,2.684915e+05,6.726667,43599.666667,45048.428571,37.738950,-122.415201
Buena Vista,26.5,2013.5,4.879878e+05,5.866667,53112.333333,54370.142857,37.806504,-122.420778
Corona Heights,36.5,2013.5,3.941081e+05,4.850000,52034.000000,53150.428571,37.761812,-122.443185
Cow Hollow,46.5,2013.5,6.144821e+05,5.825000,63147.833333,64405.571429,37.798033,-122.439646
Crocker Amazon,56.5,2013.5,1.906271e+05,5.543333,36570.500000,37788.714286,37.711952,-122.437584
Diamond Heights,66.5,2013.5,2.655130e+05,5.900000,44569.833333,45690.428571,37.742372,-122.442477
Downtown,76.5,2013.5,4.544174e+05,5.076667,39374.166667,39998.857143,37.794574,-122.399944
Excelsior,86.5,2013.5,3.420820e+05,5.148333,36686.500000,37907.000000,37.724415,-122.427229


In [23]:
#combined.to_csv('combine.csv', sep=',', encoding='utf-8')
merged_csv.to_csv('hp_and_rent.csv', sep=',', encoding='utf-8')
merged_gb.to_csv('hp_and_rent_grouped.csv', sep=',', encoding='utf-8')

In [65]:
merged_csv

,Unnamed: 0,Neighborhood,Year,Assessed Land Value,Pct Change,Region Type,Data Type,Yearly Rent Price,Avg Rent Rate,City,Lat,Lng
0,4,Bayview,2011,175691.806133,0.14,neighborhood,All Homes,30723,37087.571429,San Francisco,37.730416,-122.384424
1,5,Bayview,2012,177818.914467,1.21,neighborhood,All Homes,28821,37087.571429,San Francisco,37.730416,-122.384424
2,6,Bayview,2013,182203.741400,2.47,neighborhood,All Homes,30433,37087.571429,San Francisco,37.730416,-122.384424
3,7,Bayview,2014,185691.451367,1.91,neighborhood,All Homes,35338,37087.571429,San Francisco,37.730416,-122.384424
4,8,Bayview,2015,198571.039233,6.94,neighborhood,All Homes,42870,37087.571429,San Francisco,37.730416,-122.384424
5,9,Bayview,2016,223100.197133,12.35,neighborhood,All Homes,45681,37087.571429,San Francisco,37.730416,-122.384424
6,14,Bernal Heights,2011,228865.782800,3.96,neighborhood,All Homes,34471,45048.428571,San Francisco,37.738950,-122.415201
7,15,Bernal Heights,2012,241607.198400,5.57,neighborhood,All Homes,35739,45048.428571,San Francisco,37.738950,-122.415201
8,16,Bernal Heights,2013,252609.859300,4.55,neighborhood,All Homes,38924,45048.428571,San Francisco,37.738950,-122.415201
9,17,Bernal Heights,2014,270243.422800,6.98,neighborhood,All Homes,43654,45048.428571,San Francisco,37.738950,-122.415201


In [24]:
import requests
import json
import gmaps
import gmaps.datasets

In [35]:
#Creating a HeatMap to show Neighborhoods Rent Rates  
locations = merged_gb[["Lat", "Lng"]]
weights = merged_gb["Assessed Land Value"]
weights2 = merged_gb["Avg Rent Rate"]
fig = gmaps.figure(map_type="SATELLITE")
fig.add_layer(gmaps.heatmap_layer(locations, weights=weights, point_radius = 25, gradient = ['aqua', 'purple']))
fig.add_layer(gmaps.heatmap_layer(locations, weights=weights2, point_radius = 25))
fig

Figure(layout=FigureLayout(height='420px'))

In [36]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [55]:
data = pd.read_csv("hp_and_rent_grouped.csv", encoding='utf8', engine='python')
data.head()

,Neighborhood,Unnamed: 0,Year,Assessed Land Value,Pct Change,Yearly Rent Price,Avg Rent Rate,Lat,Lng
0,Bayview,6.5,2013.5,190512.858289,4.170000,35644.333333,37087.571429,37.730416,-122.384424
1,Bernal Heights,16.5,2013.5,268491.454817,6.726667,43599.666667,45048.428571,37.738950,-122.415201
2,Buena Vista,26.5,2013.5,487987.841867,5.866667,53112.333333,54370.142857,37.806505,-122.420778
3,Corona Heights,36.5,2013.5,394108.123417,4.850000,52034.000000,53150.428571,37.761812,-122.443185
4,Cow Hollow,46.5,2013.5,614482.066967,5.825000,63147.833333,64405.571429,37.798033,-122.439646


In [61]:
airbnb_path = "../Data/Airbnb_listings.csv"
airbnb_df = pd.read_csv(airbnb_path, encoding='utf8', engine='python')
airbnb_df["neighbourhood"].value_counts()

Mission                  27934
Western Addition         20746
South of Market          17289
Castro/Upper Market      13250
Downtown/Civic Center    12124
Bernal Heights           11805
Haight Ashbury           11226
Noe Valley                9814
Nob Hill                  8292
Marina                    8273
Inner Richmond            7469
Potrero Hill              7362
Outer Sunset              6706
Russian Hill              6291
Pacific Heights           6276
Outer Richmond            5108
North Beach               4746
Outer Mission             4556
Inner Sunset              4334
Financial District        3818
Excelsior                 3780
Parkside                  3609
Chinatown                 3375
Bayview                   3352
West of Twin Peaks        3327
Ocean View                2942
Twin Peaks                2433
Glen Park                 2142
Lakeshore                 1483
Visitacion Valley         1195
Presidio Heights          1127
Crocker Amazon             854
Seacliff

In [64]:
airbnb_df

,Unnamed: 0,date,neighbourhood,property type,daily rate,annual rate (USD)
0,0,2016-10,Seacliff,House,$105.00,38325.0
1,1,2016-10,Seacliff,House,$300.00,109500.0
2,2,2016-10,Seacliff,Apartment,$175.00,63875.0
3,3,2016-10,Seacliff,House,$90.00,32850.0
4,4,2016-10,Seacliff,Condominium,$400.00,146000.0
5,5,2016-10,Seacliff,Apartment,$85.00,31025.0
6,6,2016-10,Seacliff,House,$195.00,71175.0
7,7,2016-10,Seacliff,Apartment,$195.00,71175.0
8,8,2016-10,Seacliff,House,$135.00,49275.0
9,9,2016-10,Seacliff,House,$111.00,40515.0


In [62]:
merged_test = pd.merge(merged_gb,airbnb_df, 
                  how='inner',
                  left_on=["Neighborhood"],
                  right_on=["neighbourhood"])

In [63]:
merged_test

,Unnamed: 0_x,Year,Assessed Land Value,Pct Change,Yearly Rent Price,Avg Rent Rate,Lat,Lng,Unnamed: 0_y,date,neighbourhood,property type,daily rate,annual rate (USD)
0,6.5,2013.5,190512.858289,4.170,35644.333333,37087.571429,37.730416,-122.384424,4516,2016-10,Bayview,House,$92.00,33580.0
1,6.5,2013.5,190512.858289,4.170,35644.333333,37087.571429,37.730416,-122.384424,4517,2016-10,Bayview,Townhouse,$80.00,29200.0
2,6.5,2013.5,190512.858289,4.170,35644.333333,37087.571429,37.730416,-122.384424,4518,2016-10,Bayview,House,$75.00,27375.0
3,6.5,2013.5,190512.858289,4.170,35644.333333,37087.571429,37.730416,-122.384424,4519,2016-10,Bayview,House,$149.00,54385.0
4,6.5,2013.5,190512.858289,4.170,35644.333333,37087.571429,37.730416,-122.384424,4520,2016-10,Bayview,House,$136.00,49640.0
5,6.5,2013.5,190512.858289,4.170,35644.333333,37087.571429,37.730416,-122.384424,4521,2016-10,Bayview,House,$500.00,182500.0
6,6.5,2013.5,190512.858289,4.170,35644.333333,37087.571429,37.730416,-122.384424,4522,2016-10,Bayview,Apartment,$59.00,21535.0
7,6.5,2013.5,190512.858289,4.170,35644.333333,37087.571429,37.730416,-122.384424,4523,2016-10,Bayview,House,$189.00,68985.0
8,6.5,2013.5,190512.858289,4.170,35644.333333,37087.571429,37.730416,-122.384424,4524,2016-10,Bayview,House,$50.00,18250.0
9,6.5,2013.5,190512.858289,4.170,35644.333333,37087.571429,37.730416,-122.384424,4525,2016-10,Bayview,House,$200.00,73000.0
